In [2]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import re
import time
import numpy as np
import gc
#import xgboost as xgb
#import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import math
import pickle
import os

#from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from scipy.stats import randint as sp_randint
from sklearn.model_selection import KFold, StratifiedKFold
#from prettytable import PrettyTable
from sklearn.metrics import roc_curve,auc, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split, cross_validate, RandomizedSearchCV, GridSearchCV
from scipy.stats import uniform
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import normalize
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, log_loss
from sklearn.metrics import fbeta_score
from imblearn import over_sampling
from sklearn.linear_model import SGDClassifier
from collections import Counter
from scipy.sparse import hstack
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from datetime import datetime
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier

ModuleNotFoundError: No module named 'matplotlib.artist'

# Data Exploration & Preparation

In [ ]:
df = pd.read_csv('loan_data_2007_2014.csv')

In [ ]:
df.sample(10)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
print("Total Columns :", df.shape[1])
print("Total Rows :", df.shape[0])

Data pinjaman memiliki dimensi awal sebesar **466285 rows** dan **75 columns**

## Duplicate Value

In [ ]:
df.drop('id', axis=1, inplace=True)
df.duplicated().sum()

Data tidak memiliki row yang duplicate

In [ ]:
df_copy = df.copy()

## Null Value

In [ ]:
total_null = df_copy.isnull().sum() 
perc_null = df_copy.isnull().sum() * 100/ len(df) 


df_null_val = pd.DataFrame({'Total Null': total_null, 'Percentage Null': perc_null}) 
df_null_val.sort_values('Percentage Null', ascending = False,inplace = True) 
null_value = df_null_val[df_null_val['Percentage Null']>0].reset_index() 
null_value

In [ ]:
# drop null di atas 40% karena dapat merusak data
col_null40 = df_null_val.loc[df_null_val['Percentage Null']> 40].index.tolist()
df_copy.drop(columns=col_null40, inplace = True) 

# null di kolom numerical diganti dengan Median
for col in df_copy.select_dtypes(exclude='object'):
    df_copy[col] = df_copy[col].fillna(df_copy[col].median())

# null di kolom categorical diganti dengan Modus
for col in df_copy.select_dtypes(include='object'):
    df_copy[col] = df_copy[col].fillna(df_copy[col].mode().iloc[0])
    
df_copy.isnull().sum()

In [ ]:
df_copy['loan_status'].value_counts()

In [ ]:
# gabungkan kategori
df_copy['loan_status'] = df_copy['loan_status'].replace({
    'Does not meet the credit policy. Status:Fully Paid': 'Fully Paid',
    'Does not meet the credit policy. Status:Charged Off': 'Charged Off'
})

In [ ]:
df_copy['term'].unique()

In [ ]:
# hanya mengambil 3 karakter awal dari term
df_copy['term'] = df_copy['term'].apply(lambda term: int(term[:3]))

In [ ]:
# mengatur kolom bertipe datetime
df_copy['earliest_cr_line'] = pd.to_datetime(df_copy['earliest_cr_line'], format = '%b-%y') 
df_copy['last_credit_pull_d'] = pd.to_datetime(df_copy['last_credit_pull_d'], format = '%b-%y') 
df_copy['last_pymnt_d'] = pd.to_datetime(df_copy['last_pymnt_d'], format = '%b-%y') 

df_copy['earliest_cr_line'] = round(pd.to_numeric((pd.to_datetime('2016-03-01') - df_copy['earliest_cr_line']) / np.timedelta64(30, 'D'))) 
df_copy['last_credit_pull_d'] = round(pd.to_numeric((pd.to_datetime('2016-03-01') - df_copy['last_credit_pull_d']) / np.timedelta64(30, 'D'))) 
df_copy['last_pymnt_d'] = round(pd.to_numeric((pd.to_datetime('2016-03-01') - df_copy['last_pymnt_d']) / np.timedelta64(30, 'D'))) 

In [ ]:
# menghapus kolom yang tidak dibutuhkan
df_copy.drop(['Unnamed: 0', 'policy_code','application_type','title', 'url', 'addr_state','sub_grade','member_id', 'emp_title','pymnt_plan','issue_d', 'zip_code'], axis=1, inplace=True)

In [ ]:
unique_list = [] 
for col in df_copy.columns: 
    unique_list.append([col, df_copy[col].nunique(), df_copy[col].unique()[:5]]) 

unique_val = pd.DataFrame(data=unique_list, columns='Feature,Unique,Unique Sample'.split(","))
unique_val

In [ ]:
df_copy.info()

# Exploratory Data Analysis (EDA)

## Descriptive Statistics

In [ ]:
numerical = df_copy.select_dtypes(include=np.number).columns
categorical = df_copy.select_dtypes(exclude=np.number).columns

In [ ]:
df_copy[numerical].describe().T

In [ ]:
df_copy[categorical].describe().T

## Multivariate Analysis

In [ ]:
plt.figure(figsize=(25,25)) 
sns.heatmap(df_copy[numerical].corr(), cmap='viridis', annot=True, fmt='.2f') 
plt.show()

In [ ]:
# mencari kolom yang memiliki korelasi tinggi
correlation = df_copy[numerical].corr().abs() 
corr_list = correlation.where(np.triu(np.ones(correlation.shape), k=1).astype(bool)) 
high_corr = [column for column in corr_list.columns if any(corr_list[column] > 0.7)]
high_corr

In [ ]:
df_copy.drop(high_corr, axis=1, inplace=True)

## Univariate Analysis

In [ ]:
df_copy.dtypes.value_counts()

In [ ]:
univariate = df_copy.select_dtypes(include=[np.float64,np.int64])

In [ ]:
# displot untuk melihat distribusi tiap kolom numerical
plt.figure(figsize=(30, 30))
for i in range(0, 21):
    plt.subplot(7, 5, i+1)
    sns.distplot(univariate.iloc[:,i], color='green')
    plt.tight_layout()

In [ ]:
# boxplot untuk melihat kemungkinan outliers tiap kolom numerical
plt.figure(figsize=(30, 30))
for i in range(0, 21):
    plt.subplot(7, 5, i+1)
    sns.boxplot(univariate.iloc[:,i], color='green', orient='v')
    plt.xlabel(univariate.columns[i])
    plt.tight_layout()

## Transformation

In [ ]:
# Fungsi untuk menerapkan transformasi Yeo-Johnson
def apply_yeojohnson(df, column):
    pt = PowerTransformer(method='yeo-johnson')
    # Hanya menerapkan transformasi pada kolom numerik
    transformed_data = pt.fit_transform(df[[column]])
    df[column] = transformed_data
    return df

# Salinan dataset untuk menyimpan hasil transformasi
df_transformed = df_copy.copy()

# Menerapkan transformasi Yeo-Johnson pada setiap kolom numerik di X_test
for column in univariate:
    df_transformed = apply_yeojohnson(df_transformed, column)

# Fungsi untuk membuat histogram dan boxplot
def plot_transformation(transformed, column):
    fig, ax = plt.subplots(1, 2, figsize=(10, 4))

    # Histogram Transformed
    sns.histplot(transformed[column], kde=True, ax=ax[0], color='green')
    ax[0].set_title(f'Transformed (Yeo-Johnson) - {column}')
    ax[0].set_xlabel(f'log_{column}')

    # Boxplot Transformed
    sns.boxplot(x=transformed[column], ax=ax[1], color='green')
    ax[1].set_title(f'Transformed (Yeo-Johnson) - {column}')
    ax[1].set_xlabel(f'log_{column}')

    plt.tight_layout()
    plt.show()

# Menampilkan plot untuk setiap kolom yang telah ditransformasi
for column in univariate:
    plot_transformation(df_transformed, column)

## Bivariate Analysis

In [ ]:
eda = df_copy.copy()

In [ ]:
loan_status_counts = eda['loan_status'].value_counts().sort_values(ascending=True)

fig = plt.figure(figsize=(12, 7)) 
loan_status_counts.plot(kind='barh', color='green', width=0.8) 

# Menambahkan judul dan label sumbu
plt.title('Jumlah Pengajuan berdasarkan Loan Status\n', fontsize=14) 
plt.ylabel('Loan Status') 
plt.xlabel('Applicants')

for i, v in enumerate(loan_status_counts):
    plt.text(v + 3, i + .25, str(v), color='black')

# Menampilkan plot
plt.show()

In [ ]:
eda['loan_status'] = eda['loan_status'].astype(str)
eda['term'] = eda['term'].astype(str)

plt.figure(figsize=(15, 10))
fig = sns.countplot(data=eda, x='loan_status', hue='term', palette='viridis')

plt.title('Loan Status by Term\n', fontsize=12)
plt.xlabel('\nLoan Status', fontsize=12)
plt.ylabel('Count\n', fontsize=12)

for p in fig.patches:
    height = p.get_height() 
    if height > 0: 
        fig.annotate(f'{int(height)}',
                     (p.get_x() + p.get_width() / 2, height),
                     ha='center', va='bottom', fontsize=10)

plt.show()

In [ ]:

plt.figure(figsize=(15, 10))
fig = sns.countplot(data=eda, x='loan_status', hue='verification_status', palette='viridis')

plt.title('Loan Status by Verification Status\n', fontsize=12)
plt.xlabel('\nLoan Status', fontsize=12)
plt.ylabel('Count\n', fontsize=12)

for p in fig.patches:
    height = p.get_height() 
    if height > 0: 
        fig.annotate(f'{int(height)}',
                     (p.get_x() + p.get_width() / 2, height),
                     ha='center', va='bottom', fontsize=10)

plt.show()

# Feature Engineering

In [ ]:
df_feature = df_transformed.copy()

In [ ]:
# memberikan label pada target
df_feature['binary_loan_status'] = df_feature['loan_status'].apply(
    lambda x: 1 if x in ['Fully Paid', 'Current', 'In Grace Period'] else 0 # label 1 jika berstatus GOOD dan 0 jika berstatus BAD
)

## Feature Encoding

In [ ]:
# drop feature categorical dengan unique value yang terlalu banyak
df_feature.drop(['emp_length','purpose'], axis = 1, inplace=True)

In [ ]:
for cat in ['grade', 'verification_status', 'home_ownership', 'initial_list_status']:
  onehots = pd.get_dummies(df_feature[cat], prefix=cat)
  df_feature = df_feature.join(onehots)

df_feature = df_feature.drop(columns=['grade', 'verification_status', 'home_ownership', 'initial_list_status'], axis =1)

# mengubah hasil OHE menjadi int
boolean_cols = df_feature.select_dtypes(include=['bool']).columns
df_feature[boolean_cols] = df_feature[boolean_cols].astype(int)

df_feature.sample(5)

In [ ]:
print("Total Rows :", df_feature.shape[0])
print("Total Features :", df_feature.shape[1])

In [ ]:
df_feature.dtypes.value_counts()

In [ ]:
df_feature.columns.tolist()

# Modelling

In [ ]:
df_final = df_feature.copy()

In [ ]:
X = df_final.drop(labels=['binary_loan_status','loan_status'],axis=1)
y = df_final[['binary_loan_status']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify=y, random_state = 42)

In [ ]:
scaler = StandardScaler()

X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(scaler.fit_transform(X_test), columns=X_test.columns)

In [ ]:
y.value_counts()

In [ ]:
X_train, y_train = over_sampling.SMOTE(sampling_strategy=1).fit_resample(X_train, y_train)

In [ ]:
y_train.value_counts()

In [ ]:
# Untuk evaluasi model 

def model_evaluation(y_true, y_pred, y_pred_proba, model_name):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    f2 = fbeta_score(y_true, y_pred, beta=2.0)
    auc = roc_auc_score(y_true, y_pred_proba[:, 1])
    
    results = pd.DataFrame([[model_name, acc, prec, rec, f1, f2, auc]],
                       columns=["Model", "Accuracy", "Precision", "Recall",
                                "F1 Score", "F2 Score", "roc_auc_score"])
    return results

In [ ]:
# Untuk cek overfitting vs underfitting atau best fit
def check_model_fit(classifier, X_train, X_val, y_train, y_val, model_name="Model"):
    """
    Membandingkan performa model pada data training dan validation
    untuk mendeteksi overfitting/underfitting
    """
    # Prediksi pada data training
    y_train_pred = classifier.predict(X_train)
    y_train_pred_proba = classifier.predict_proba(X_train)
    
    # Prediksi pada data validation
    y_test_pred = classifier.predict(X_test)
    y_test_pred_proba = classifier.predict_proba(X_test)
    
    # Metrics untuk training
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_precision = precision_score(y_train, y_train_pred)
    train_recall = recall_score(y_train, y_train_pred)
    train_f1 = f1_score(y_train, y_train_pred)
    train_f2 = fbeta_score(y_train, y_train_pred, beta=2.0)
    train_roc_auc = roc_auc_score(y_train, y_train_pred_proba[:, 1])

    # Metrics untuk validation
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_precision = precision_score(y_test, y_test_pred)
    test_recall = recall_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred)
    test_f2 = fbeta_score(y_test, y_test_pred, beta=2.0)
    test_roc_auc = roc_auc_score(y_test, y_test_pred_proba[:, 1])
    

    # Membuat DataFrame perbandingan
    comparison = pd.DataFrame({
        'Metric': ['Accuracy', 'Precision', 'Recall', 'F1', 'F2', 'ROC AUC'],
        'Training': [train_accuracy, train_precision, train_recall, train_f1, train_f2, train_roc_auc],
        'Validation': [test_accuracy, test_precision, test_recall, test_f1, test_f2, test_roc_auc],
        'Difference': [train_accuracy - test_accuracy, 
                      train_precision - test_precision,
                      train_recall - test_recall,
                      train_f1 - test_f1,
                      train_f2 - test_f2,
                      train_roc_auc - test_roc_auc]
    })
    
    print(f"\nModel Fit Analysis for {model_name}:")
    print(comparison.round(4))
    
    # Analisis overfitting/underfitting
    avg_diff = comparison['Difference'].mean()

In [ ]:
def plot_confusion_matrix(model, X_test, y_test, title):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    fig, ax = plt.subplots(1, 1, figsize=(5,5))
    sns.heatmap(cm, annot=True, fmt='g', cmap='Greens', annot_kws={"fontsize":12}, ax=ax)

    # setting title and axis labels
    plt.tick_params(axis='both', which='major', labelsize=12)
    ax.set_xlabel('Predicted Labels', fontsize=12)
    ax.set_ylabel('True Labels', fontsize=12)
    ax.set_title('Confusion Matrix ' + title, fontsize=14)
    ax.xaxis.set_ticklabels(['Bad', 'Good'])
    ax.yaxis.set_ticklabels(['Bad', 'Good'])
    plt.tight_layout()
    plt.show()

## Logistic Regression

In [ ]:
classifier_lr = LogisticRegression()
classifier_lr.fit(X_train, y_train)
y_pred = classifier_lr.predict(X_test)
y_pred_proba = classifier_lr.predict_proba(X_test)

# Evaluasi
results_lr = model_evaluation(y_test, y_pred, y_pred_proba, "Logistic Regression")
results_lr

In [ ]:
check_model_fit(
    classifier_lr,
    X_train,  # Data training
    X_test,   # Data validation
    y_train,  # Label training
    y_test,   # Label validation
    "Logistic Regression"
)

In [ ]:
plot_confusion_matrix(classifier_lr, X_test, y_test, title='Logistic Regression')
classifier_lr.get_params()

## Decision Tree

In [ ]:
classifier_dt = DecisionTreeClassifier()
classifier_dt.fit(X_train, y_train)
y_pred = classifier_dt.predict(X_test)
y_pred_proba = classifier_dt.predict_proba(X_test)

# Evaluasi
results_dt = model_evaluation(y_test, y_pred, y_pred_proba, "Decision Tree")
results_dt

In [ ]:
check_model_fit(
    classifier_dt,
    X_train,  # Data training
    X_test,   # Data validation
    y_train,  # Label training
    y_test,   # Label validation
    "Descision Tree"
)

## Random Forest

In [ ]:
classifier_rf = RandomForestClassifier()
classifier_rf.fit(X_train, y_train)
y_pred = classifier_rf.predict(X_test)
y_pred_proba = classifier_rf.predict_proba(X_test)

# Evaluasi
results_rf = model_evaluation(y_test, y_pred, y_pred_proba, "Random Forest")
results_rf

In [ ]:
check_model_fit(
    classifier_rf,
    X_train,  # Data training
    X_test,   # Data validation
    y_train,  # Label training
    y_test,   # Label validation
    "Random Forest"
)

## LightGBM

In [ ]:
classifier_lgbm = LGBMClassifier()
classifier_lgbm.fit(X_train, y_train)
y_pred = classifier_lgbm.predict(X_test)
y_pred_proba = classifier_lgbm.predict_proba(X_test)

# Evaluasi
results_lgbm = model_evaluation(y_test, y_pred, y_pred_proba, "LightGBM")
results_lgbm

In [ ]:
check_model_fit(
    classifier_lgbm,
    X_train,  # Data training
    X_test,   # Data validation
    y_train,  # Label training
    y_test,   # Label validation
    "LightGBM"
)

## Ada Boost

In [ ]:
classifier_ab = AdaBoostClassifier()
classifier_ab.fit(X_train, y_train)
y_pred = classifier_ab.predict(X_test)
y_pred_proba = classifier_ab.predict_proba(X_test)

# Evaluasi
results_ab = model_evaluation(y_test, y_pred, y_pred_proba, "Ada Boost")
results_ab

In [ ]:
check_model_fit(
    classifier_ab,
    X_train,  # Data training
    X_test,   # Data validation
    y_train,  # Label training
    y_test,   # Label validation
    "Ada Boost"
)

## XGBoost

In [ ]:
classifier_xgb =  XGBClassifier()
classifier_xgb.fit(X_train, y_train)
y_pred = classifier_xgb.predict(X_test)
y_pred_proba = classifier_xgb.predict_proba(X_test)

# Evaluasi
results_xgb = model_evaluation(y_test, y_pred, y_pred_proba, "Ada Boost")
results_xgb

In [ ]:
check_model_fit(
    classifier_xgb,
    X_train,  # Data training
    X_test,   # Data validation
    y_train,  # Label training
    y_test,   # Label validation
    "XGBoost"
)